In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import glob

In [2]:
def create_ds(ds, array):
    lonn = ds.lon.values
    latt = ds.lat.values

    ds_ = xr.Dataset({'s': ([ 'lat', 'lon'], array)},
                    coords={'lon': (['lon'], lonn),
                            'lat': (['lat'], latt),})
    return ds_

In [3]:
import numpy as np
import scipy.stats as stats

def calculate_ccscale_slope(temperature_data, precipitation_data):
    # Ensure the data is flat (1D)
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan

    # Remove data points where either temperature or precipitation contains NaN
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan
    
    # Convert Kelvin temperatures to Celsius
    temperature_data_celsius = temperature_data - 273.15

    # Set the temperature bin size and sliding step
    bin_size = 1.0  # Bin size
    step = 0.5      # Sliding step
    min_temp = temperature_data_celsius.min()
    max_temp = temperature_data_celsius.max()
    
    
    # Create sliding temperature bins
    temperature_bins = np.arange(min_temp, max_temp, step)
    overlapping_bins = [(start, start + bin_size) for start in temperature_bins]

    # Collect precipitation data for each temperature bin using a dictionary
    precipitation_per_bin = {i: [] for i in range(len(overlapping_bins))}
    for temp, precip in zip(temperature_data_celsius, precipitation_data):
        for i, (bin_start, bin_end) in enumerate(overlapping_bins):
            if bin_start <= temp < bin_end:
                precipitation_per_bin[i].append(precip)

    # Calculate the 99th percentile of log precipitation, mean temperature, and confidence intervals
    log_precipitation_99 = []
    mean_temperatures = []
    for idx, (bin_start, bin_end) in enumerate(overlapping_bins):
        bin_data = precipitation_per_bin[idx]
        if len(bin_data) >= 80:
            log_precip = np.log(bin_data)
            quantile_99 = np.percentile(log_precip, 99)
            mean_temp = (bin_start + bin_end) / 2
            log_precipitation_99.append(quantile_99)
            mean_temperatures.append(mean_temp)

    # Perform linear regression if we have at least 5 bins with sufficient data
    if len(log_precipitation_99) >= 5:
        slope, intercept, r_value, p_value, std_err = stats.linregress(mean_temperatures, log_precipitation_99)
    else:
        slope = np.nan
        intercept = np.nan

    return mean_temperatures, log_precipitation_99, slope, intercept

# Example usage:
# temperature_data = np.array([...])
# precipitation_data = np.array([...])
# results = calculate_ccscale_slope(temperature_data, precipitation_data)
# print(results)



In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def calculate_ccscale_quantreg(temperature_data, precipitation_data):
    # 确保数据是平的（一维的）
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # 移除温度或降水量中包含NaN的数据点
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan

    # 将开尔文温度转换为摄氏度
    temperature_data_celsius = temperature_data - 273.15

    # 创建一个 DataFrame 来包含温度和降水量的数据
    data = pd.DataFrame({
        'Temperature': temperature_data_celsius,
        'Precipitation': precipitation_data
    })

    # 定义模型：在 0.99 分位数处进行分位数回归
    quantile_model = sm.QuantReg(np.log(data['Precipitation']), sm.add_constant(data['Temperature']))
    quantile_regression_result = quantile_model.fit(q=0.99).params[1]

    return quantile_regression_result

# 使用实际的 temperature_data 和 precipitation_data 调用该函数
# 例如：
# result = calculate_ccscale_quantreg(temperature_data, precipitation_data)
# print(result.summary())


In [5]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'

In [ ]:
folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
            slope_nt = calculate_ccscale_slope(temperature_data_nt, precipitation_data_nt)[2]
            slope_dt = calculate_ccscale_slope(temperature_data_dt, precipitation_data_dt)[2]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_lrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_lrs_dt = create_ds(ds_t, arr_slope_dt)
            ds_lrs_nt.to_netcdf(input_folder_t+'ds_lrs_nt'+folder+'.nc')
            ds_lrs_dt.to_netcdf(input_folder_t+'ds_lrs_dt'+folder+'.nc')

In [6]:
folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt)
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt)
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
            ds_qrs_nt.to_netcdf(input_folder_t+'ds_qrs_nt'+folder+'.nc')
            ds_qrs_dt.to_netcdf(input_folder_t+'ds_qrs_dt'+folder+'.nc')

/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

In [ ]:
folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt)
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt)
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_qrs_nt.to_netcdf(input_folder_t+'ds_dewqrs_nt'+folder+'.nc')
    ds_qrs_dt.to_netcdf(input_folder_t+'ds_dewqrs_dt'+folder+'.nc')

/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

In [5]:
ds

<xarray.Dataset> Size: 2MB
Dimensions:  (time: 24, lat: 78, lon: 243)
Coordinates:
  * lon      (lon) float64 2kB -124.8 -124.8 -124.7 ... -112.6 -112.6 -112.5
  * lat      (lat) float64 624B 41.08 41.11 41.15 41.18 ... 43.73 43.76 43.8
  * time     (time) datetime64[ns] 192B 2019-06-24 ... 2019-06-24T23:00:00
Data variables:
    t2       (time, lat, lon) float32 2MB ...

In [13]:
import datetime
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
US = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata'
output1 = '/N/project/Zli_lab/gongg/CONUS404_data/LST/'
ds_raster = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2022-09-30_23:00:00.nc')
# 定义所有的U*前缀
prefixes = [
    'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88'
]

for year in range(1990, 2000):  # 从1980年到2022年
    for month in [6, 7, 8]:  # 只读取6, 7, 8月的数据
        print(datetime.datetime.now().time())
        days_in_month = 30 if month == 6 else 31  # 6月30天，7月和8月31天
        
        
        for day in range(1, days_in_month + 1):
            files_to_open = []
            # 对每一个前缀和日期组合构造文件路径
            for prefix in prefixes:
                file_pattern = f'{base_path}/{prefix}_mt_{year}_{month:02d}_{day:02d}.nc'
                files_to_open.append(file_pattern)
                
            ds = xr.open_mfdataset(files_to_open)
            lon = ds_raster['XLONG'].values
            lat = ds_raster['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_raster['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_raster['PREC_ACC_NC'].dims[1:], coords={'south_north': ds_raster['PREC_ACC_NC'].coords['south_north'], 'west_east': ds_raster['PREC_ACC_NC'].coords['west_east']})
            ds_conus = ds_raster.where(mask_da, drop=True)

            XLON = ds_conus.XLONG.values[:707,:]
            XLAT = ds_conus.XLAT.values[:707,:]
            ds_n = ds.assign_coords({
                'XLON': (('lat', 'lon'), XLON),
                'XLAT': (('lat', 'lon'), XLAT)
            })

            regions_dict = {
                'NE': ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'WV'],
                'Midwest': ['IA', 'MI', 'MN', 'WI', 'IL', 'IN', 'MO', 'OH'],
                'SE': ['AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 'KY', 'AR', 'LA', 'MS'],
                'NGP': ['MT', 'NE', 'ND', 'SD', 'WY'],
                'SGP': ['KS', 'OK', 'TX'],
                'SW': ['AZ', 'CO', 'NM', 'UT', 'CA', 'NV'],
                'NW': ['ID', 'OR', 'WA']
            }
            regions = {name: US[US['STUSPS'].isin(states)] for name, states in regions_dict.items()}
            regi = ['NE','Midwest','SE','NGP','SGP','SW','NW',]

            ds_results = {}

            for r in regi:
                lon = ds_n['XLON'].values
                lat = ds_n['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )

                grid.set_crs(regions[r].crs, inplace=True)
                grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')

                mask = np.full((ds_n['t2'].shape[1], ds_n['t2'].shape[2]), False) 
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)
                    mask[row, col] = True 

                mask_da = xr.DataArray(
                    mask, 
                    dims=['lat', 'lon'],
                    coords={
                        'lat': ds_n['lat'].values,
                        'lon': ds_n['lon'].values
                    }
                )

                ds_ssss = ds_n.where(mask_da, drop=True)
                ds_results[f'ds_{r}'] = ds_ssss

            ds_results['ds_NE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NE/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_Midwest'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}Midwest/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SE/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NGP/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SGP/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SW/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NW/mt_{year}_{month:02d}_{day:02d}.nc")

16:36:39.714122
16:46:44.741640
16:58:07.278625


FileNotFoundError: [Errno 2] No such file or directory: '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata/U-58_mt_1990_08_07.nc'